### Ler os arquivos Parquet que foram salvos

In [0]:
from pyspark.sql.functions import col, to_date, when,first,round

In [0]:
df = spark.read.parquet("dbfs:/databricks-results/bronze/*/*/*")


Selecionando somente as moedas relevantes para nosso projeto.


In [0]:
moedas = ['USD','EUR','GBP']

df_moedas = df.filter(df.moeda.isin(moedas))

In [0]:
df_moedas.show(5)

In [0]:
df_moedas.printSchema()

In [0]:
df_moedas = df_moedas.withColumn("data", to_date(col("data"), "yyyy/MM/dd"))

In [0]:
df_moedas.printSchema()

In [0]:
df_moedas.show(5)

In [0]:
resultado_taxas_conversao = (
    df_moedas
    .groupBy("data")
    .pivot("moeda")
    .agg(first("taxa"))
    .orderBy("data", ascending=False)
)

resultado_taxas_conversao.show(5)

In [0]:
resultado_valores_reais = resultado_taxas_conversao.select("*")

In [0]:
for moeda in moedas:
    resultado_valores_reais = resultado_valores_reais.withColumn(moeda, round(1/col(moeda),3 ))

In [0]:
resultado_taxas_conversao = resultado_taxas_conversao.coalesce(1)
resultado_valores_reais = resultado_valores_reais.coalesce(1)

In [0]:
resultado_taxas_conversao.write\
    .mode("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/taxas_conversao")

resultado_valores_reais.write\
    .mode("overwrite")\
    .format("csv")\
    .option("header", "true")\
    .save("dbfs:/databricks-results/prata/valores_reais")